# Client Churn Prediction
### CRISP-DM Cycle 3
---

The Top Bank company operates in Europe with a bank account as the main product, this product can keep client's salary and make payments. This account doesn't have any cost in the first 12 months, however, after that time trial, the client needs to rehire the bank for upcoming 12 months and redo this process every year. Recently the Analytics Team noticed that the churn rate is increasing.

As a Data Science Consultant, you need to create an action plan to decrease the number of churn customers and show the financial return on your solution.
At the end of your consultancy, you need to deliver to the TopBottom CEO a model in production, which will receive a customer base via API and return that same base with an extra column with the probability of each customer going into churn.
In addition, you will need to provide a report reporting your model's performance and the financial impact of your solution. Questions that the CEO and the Analytics team would like to see in their report:

1.  What is Top Bank's current Churn rate?
2.  How does the churn rate vary monthly?
3.  What is the performance of the model in classifying customers as churns
4.  What is the expected return, in terms of revenue, if the company uses its model to avoid churn from customers?

> Disclaimer: This is a fictional bussiness case

## 0. PREPARATION

### 0.1 Planning

#### Input

- Predict wheter customer will be in churn;
- Dataset with sales records and customers info.

#### Output

- Which customer will be in churn;
- Churn rate of the company;
- Performance of the model;
- Action plan


#### Tasks

1. Which customer will be in churn:
    - What is the criterion?
        - Downtime
        - Time remaining until the contract ends


2. Current churn rate of the company:
    - Calculate churn rate
    - Calculate monthly churn rate and variation

3. Performance of the model:
    - Precision at K score
    - Recall at K score

4. Action plan:
    - Discount?
    - Voucher?
    - Deposit bonus?

### 0.2 Settings

In [1]:
# Settings imports
import os
import sys

from dotenv import load_dotenv

# load .env file
env_path = "../.env"
load_dotenv(dotenv_path=env_path)

# add home path to sys.path
path = os.getenv("HOMEPATH")
sys.path.append(path)

In [2]:
# import classes
from helpers.Queries import DuckQueries
from helpers.FeatureEngineering import FeatureEngineering

# import libraries

In [3]:
seed = 42

### 0.4 Data

This dataset is avaliable [here](https://www.kaggle.com/mervetorkan/churndataset).


**Data fields**

- **RowNumber**: the number of the columns
- **CustomerID**: unique identifier of clients
- **Surname**: client's last name
- **CreditScore**: clien'ts credit score for the financial market
- **Geography**: the country of the client
- **Gender**: the gender of the client
- **Age**: the client's age
- **Tenure**: number of years the client is in the bank 
- **Balance**: the amount that the client has in their account 
- **NumOfProducts**: the number of products that the client bought 
- **HasCrCard**: if the client has a credit card 
- **IsActiveMember**: if the client is active (within the last 12 months) 
- **EstimateSalary**: estimative of anual salary of clients 
- **Exited**: if the client is a churn (*target variable*)

## 1. DATA DESCRIPTION

### 1.1 First Look

In [4]:
qb = DuckQueries()
conn = qb.get_connection(path + "/data/interim/churn.db")

query = qb.select("*").from_table("churn").limit(5).build()
result = conn.execute(query).df()
print(result.T)

                          0          1          2         3          4
row_number                1          2          3         4          5
customer_id        15634602   15647311   15619304  15701354   15737888
surname            Hargrave       Hill       Onio      Boni   Mitchell
credit_score            619        608        502       699        850
geography            France      Spain     France    France      Spain
gender               Female     Female     Female    Female     Female
age                      42         41         42        39         43
tenure                    2          1          8         1          2
balance                 0.0   83807.86   159660.8       0.0  125510.82
num_of_products           1          1          3         2          1
has_cr_card               1          0          1         0          1
is_active_member          1          1          0         0          1
estimated_salary  101348.88  112542.58  113931.57  93826.63    79084.1
exited

### 1.2 Data Dimensions

In [5]:
query = qb.shape("churn").build()
result = conn.execute(query).df()

rows = result.loc[0, "row_count"]
columns = result.loc[0, "column_count"]

print(f"Number of rows: {rows} \nNumber of columns: {columns}")

Number of rows: 10000 
Number of columns: 14


### 1.3 Check NA

In [6]:
query = qb.count_nulls_from_table("churn").build()
fetch = query = conn.execute(query).fetchone()[0]
result = conn.execute(f"SELECT {fetch} FROM churn").df()
print(result.T)

                  0
row_number        0
customer_id       0
surname           0
credit_score      0
geography         0
gender            0
age               0
tenure            0
balance           0
num_of_products   0
has_cr_card       0
is_active_member  0
estimated_salary  0
exited            0


In [7]:
query = qb.count_zeros_from_table("churn").build()
fetch = query = conn.execute(query).fetchone()[0]
result = conn.execute(f"SELECT {fetch} FROM churn").df()
print(result.T)

                     0
row_number           0
customer_id          0
credit_score         0
age                  0
tenure             413
num_of_products      0
has_cr_card       2945
is_active_member  4849
exited            7963
balance           3617
estimated_salary     0


### 1.4 Data Types

In [8]:
query = qb.column_types_from_table("churn").build()
result = conn.execute(query).df()
print(result.iloc[:, 0:2])

         column_name data_type
0         row_number    BIGINT
1        customer_id    BIGINT
2            surname   VARCHAR
3       credit_score    BIGINT
4          geography   VARCHAR
5             gender   VARCHAR
6                age    BIGINT
7             tenure    BIGINT
8            balance    DOUBLE
9    num_of_products    BIGINT
10       has_cr_card    BIGINT
11  is_active_member    BIGINT
12  estimated_salary    DOUBLE
13            exited    BIGINT


### 1.5 Descriptive Statistics

In [9]:
query = qb.select("*").from_table("churn").build()
df1 = conn.execute(query).df()
conn.close()

quantitative = df1.select_dtypes(include=["int64", "float64"])
quantitative.drop(
    columns=["customer_id", "row_number", "has_cr_card", "is_active_member", "exited"],
    inplace=True,
)

qualitative = df1[["is_active_member", "exited", "geography", "has_cr_card", "gender"]]
qualitative = qualitative.astype("str")

change_types = ["is_active_member", "exited", "has_cr_card"]

for column in change_types:
    qualitative.loc[:, column] = qualitative[column].replace({"1": "yes", "0": "no"})

qualitative.head()

,is_active_member,exited,geography,has_cr_card,gender
0,yes,yes,France,yes,Female
1,yes,no,Spain,no,Female
2,no,yes,France,yes,Female
3,no,no,France,no,Female
4,yes,no,Spain,yes,Female


In [10]:
quantitative.describe(include="all")

,credit_score,age,tenure,balance,num_of_products,estimated_salary
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,650.528800,38.921800,5.012800,76485.889288,1.530200,100090.239881
std,96.653299,10.487806,2.892174,62397.405202,0.581654,57510.492818
min,350.000000,18.000000,0.000000,0.000000,1.000000,11.580000
25%,584.000000,32.000000,3.000000,0.000000,1.000000,51002.110000
50%,652.000000,37.000000,5.000000,97198.540000,1.000000,100193.915000
75%,718.000000,44.000000,7.000000,127644.240000,2.000000,149388.247500
max,850.000000,92.000000,10.000000,250898.090000,4.000000,199992.480000


In [11]:
qualitative.describe(include="all")

,is_active_member,exited,geography,has_cr_card,gender
count,10000,10000,10000,10000,10000
unique,2,2,3,2,2
top,yes,no,France,yes,Male
freq,5151,7963,5014,7055,5457


In [12]:
fe = FeatureEngineering(seed = seed)
X_train, X_test, X_val, y_train, y_test, y_val, id_train, id_test, id_val = fe.transform(df1)

In [13]:
fe.report_na(X_train)

INFO - There are no NA values in any column.


In [14]:
fe.report_na(X_test)

INFO - There are no NA values in any column.


In [15]:
fe.report_na(X_val)

INFO - There are no NA values in any column.


In [16]:
fe.save_data(X_train, path + "/data/processed/X_train.parquet")

WARNING - File already exists. It will be overwritten.
INFO - Data successfully saved
